In [2]:
"""Module containing utils"""
import os
from numpy.polynomial.legendre import Legendre
import numpy as np
import scipy.linalg as la
import pandas as pd
import glob
import nibabel as nib
import subprocess as sp

In [3]:
#Define functions
def clean_data(data, confounds):
    """Clean data by regressing out the following nuisance regressors:
    - six motion parameters and their derivatives
    - global signal
    - framewise displacement
    - six aCompCor components
    - polynomial regressors up to second order Parameters
    ----------
    data : array of shape (n_volumes, n_features)
    flattened EPI data
    confounds : pandas Dataframe
    dataframe containing confounds generated by fmriprepp
    Returns
    -------
    data_clean : array of shape (n_volumes, n_features)
    denoised data"""

    # make predictor matrix using confounds computed by fmriprep
    columns = [
    'global_signal',
    'framewise_displacement',
    'trans_x', 'trans_x_derivative1',
    'trans_y', 'trans_y_derivative1',
    'trans_z', 'trans_z_derivative1',
    'rot_x', 'rot_x_derivative1',
    'rot_y', 'rot_y_derivative1',
    'rot_z', 'rot_z_derivative1' ]
    # compcor
    n_comp_cor = 6
    columns += [f"a_comp_cor_{c:02d}" for c in range(n_comp_cor)]
    X = confounds[columns].values

    # remove nans
    X[np.isnan(X)] = 0.

    # add polynomial components
    n_samples = X.shape[0]
    X = np.hstack((X, make_poly_regressors(n_samples, order=2)))

    # time to clean up
    # center the data first and store the mean
    data_mean = data.mean(0)
    data = data - data_mean
    coef, _, _, _ = la.lstsq(X, data)
    # remove trends and add back mean of the data
    data_clean = data - X.dot(coef) + data_mean

    return data_clean

def load_confounds(subject, task, run):
    confounds = glob.glob(os.path.join(data_dir, sub, 'func', f'{subject}_{task}_{run}_desc-confounds_timeseries.tsv'))
    df = pd.read_csv(confounds[0], sep='\t')
    return df

def extract_cols(df):
    cols = ['framewise_displacement']
    return df[cols]

def make_poly_regressors(n_samples, order=2):
    # mean
    X = np.ones((n_samples, 1))
    for d in range(order):
        poly = Legendre.basis(d + 1)
        poly_trend = poly(np.linspace(-1, 1, n_samples))
        X = np.hstack((X, poly_trend[:, None]))
    return X

In [4]:
#Subject check 
subject = np.arange(1,34)
sub_sm_exclude = [8]
sub_sm = np.delete(subject, sub_sm_exclude)
#print(subject)
print(sub_sm)

[ 1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33]


In [5]:
base_dir = '/mnt/c/Users/since/Desktop/KdramaMay/'
data_dir = os.path.join(base_dir, 'data', 'derivatives')
denoised_dir = os.path.join(base_dir,'denoised', 'denoised_2024')
mask_dir = os.path.join(base_dir, 'analysis', 'Masks')
mni_mask = nib.load(os.path.join(mask_dir, 'MNI152NLin2009cAsym_mask.nii.gz')).get_fdata()
mni_image = nib.load(os.path.join(mask_dir, 'MNI152NLin2009cAsym_mask.nii.gz'))

In [12]:
for sub in sub_sm:
    if sub < 10:
        sub = '0'+str(sub)
    file_list = glob.glob(os.path.join(data_dir, f'sub-{sub}', 'func', '*preproc*.nii.gz'))
    print(len(file_list))
    for f in file_list:
        sub = os.path.basename(f).split('_')[0]
        task = os.path.basename(f).split('_')[1]
        run = os.path.basename(f).split('_')[2]
        
        print('+++++++++',sub,' ', task, ' ', run, ' start! +++++++++')
        
        print('* Step (1/4) Masking')
        #load fMRI data 
        fmri_data = nib.load(f).get_fdata()
        print('  original: ', fmri_data.shape)
        
        
        #masking fMRI data
        fmri_data = fmri_data[mni_mask==1,:].T ### -> 이거 체크
        print('  masked: ', fmri_data.shape)
        
        print('* Step (2/4) Denoising')
        fmri_compounds = load_confounds(sub, task, run)
        print(fmri_compounds.columns) # column 만 나오게 만들기 
        fmri_clean = clean_data(fmri_data, fmri_compounds)
        fmri_clean = fmri_clean.astype(np.float32)
        
        new_image = np.zeros(mni_mask.shape+(fmri_data.shape[0],), dtype=np.float32)
        new_image[mni_mask==1,:] = fmri_clean.T
        
        # save files 
        input_fname = f'{denoised_dir}/{sub}_{task}_{run}.nii.gz'
        mean_fname = f'{denoised_dir}/{sub}_{task}_{run}_1mean.nii.gz'
        mask_fname = os.path.join(mask_dir, 'MNI152NLin2009cAsym_mask.nii.gz')
        SC_fname = f'{denoised_dir}/{sub}_{task}_{run}_2scaling.nii.gz'
        
        SM_fname = f'{denoised_dir}/{sub}_{task}_{run}_3smoothing.nii.gz'
        SM_SC_fname = f'{denoised_dir}/{sub}_{task}_{run}_4scNsm.nii.gz'
        
        FWHM = 6
        nib.save(nib.Nifti1Image(new_image, mni_image.affine), input_fname)
        
        # Scaling
        print('* Step (3/4) Scaling')
        sp.call(f'3dTstat -mean -prefix {mean_fname} {input_fname}', shell=True)
        sp.call(f"3dcalc -a {input_fname} -b {mean_fname} -c {mask_fname} -expr 'c * min(200, a/b*100)*step(a)*step(b)' -prefix {SC_fname}", shell=True)

        # Spatial Smoothing
        print('* Step (4/4) Smoothing')
        sp.call(f'3dmerge -quiet -1blur_fwhm {FWHM} -doall -prefix {SM_SC_fname} {SC_fname}', shell=True)
        sp.call(f'3dmerge -quiet -1blur_fwhm {FWHM} -doall -prefix {SM_fname} {input_fname}', shell=True)
        
print('+++++++++', 'Finish', '+++++++++')

4
+++++++++ sub-01   task-encoding   run-1  start! +++++++++
* Step (1/4) Masking
  original:  (91, 109, 91, 485)
  masked:  (485, 216734)
* Step (2/4) Denoising
Index(['global_signal', 'global_signal_derivative1',
       'global_signal_derivative1_power2', 'global_signal_power2', 'csf',
       'csf_derivative1', 'csf_power2', 'csf_derivative1_power2',
       'white_matter', 'white_matter_derivative1',
       ...
       'motion_outlier106', 'motion_outlier107', 'motion_outlier108',
       'motion_outlier109', 'motion_outlier110', 'motion_outlier111',
       'motion_outlier112', 'motion_outlier113', 'motion_outlier114',
       'motion_outlier115'],
      dtype='object', length=576)
* Step (3/4) Scaling


++ 3dTstat: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ Authored by: KR Hammett & RW Cox
** ERROR: output dataset name 'sub-01_task-encoding_run-1_1mean.nii.gz' conflicts with existing file
** ERROR: dataset NOT written to disk!
++ 3dcalc: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ Authored by: A cast of thousands
** ERROR: output dataset name 'sub-01_task-encoding_run-1_2scaling.nii.gz' conflicts with existing file
** ERROR: dataset NOT written to disk!
++ 3dmerge: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1


* Step (4/4) Smoothing


** ERROR: output dataset name 'sub-01_task-encoding_run-1_4scNsm.nii.gz' conflicts with existing file
** ERROR: dataset NOT written to disk!
++ 3dmerge: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1


Program 3dmerge 


...

Program 3dmerge 
+++++++++ sub-33   task-recall   run-2  start! +++++++++
* Step (1/4) Masking
  original:  (91, 109, 91, 312)
  masked:  (312, 216734)
* Step (2/4) Denoising
Index(['global_signal', 'global_signal_derivative1', 'global_signal_power2',
       'global_signal_derivative1_power2', 'csf', 'csf_derivative1',
       'csf_power2', 'csf_derivative1_power2', 'white_matter',
       'white_matter_derivative1',
       ...
       'motion_outlier38', 'motion_outlier39', 'motion_outlier40',
       'motion_outlier41', 'motion_outlier42', 'motion_outlier43',
       'motion_outlier44', 'motion_outlier45', 'motion_outlier46',
       'motion_outlier47'],
      dtype='object', length=287)
* Step (3/4) Scaling


++ 3dTstat: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ Authored by: KR Hammett & RW Cox
++ Output dataset /mnt/c/Users/since/Desktop/KdramaMay/denoised/denoised_2024/sub-33_task-recall_run-2_1mean.nii.gz
++ 3dcalc: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset /mnt/c/Users/since/Desktop/KdramaMay/denoised/denoised_2024/sub-33_task-recall_run-2_2scaling.nii.gz
++ 3dmerge: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1


* Step (4/4) Smoothing
Program 3dmerge 


++ 3dmerge: AFNI version=AFNI_24.0.07 (Feb 15 2024) [64-bit]
++ default -1dindex = 0
++ default -1tindex = 1


Program 3dmerge 
+++++++++ Finish +++++++++
